In [1]:
import numpy as np
import torch

In [ ]:
def conv2d(image, kernel, stride=1, padding=0):
    """
    Perform a 2D convolution operation on an image using the specified kernel.

    Parameters:
    - image: 2D numpy array, the input image.
    - kernel: 2D numpy array, the convolution kernel (filter).
    - stride: int, the stride of the convolution. Default is 1.
    - padding: int, the amount of zero-padding around the image. Default is 0.

    Returns:
    - output: 2D numpy array, the result of applying the convolution.
    """

    # Add zero-padding to the image if required
    if padding > 0:
        image = np.pad(image, ((padding, padding), (padding, padding)), mode='constant')

    # Get the dimensions of the image and the kernel
    i_h, i_w = image.shape
    k_h, k_w = kernel.shape

    # Calculate the dimensions of the output
    o_h = (i_h - k_h) // stride + 1
    o_w = (i_w - k_w) // stride + 1

    # Initialize the output image
    output = np.zeros((o_h, o_w))

    # Perform the convolution
    for y in range(0, o_h):
        for x in range(0, o_w):
            # Extract the region of interest
            region = image[y * stride:y * stride + k_h, x * stride:x * stride + k_w]

            # Element-wise multiplication and summation
            output[y, x] = np.sum(region * kernel)

    return output

In [ ]:
W = np.array(
    [
        [1,0,-1],
        [2,0,-2],
        [1,0,-1]
    ]).astype(np.float32)

X = np.array(
    [
        [1,1,1,2,3],
        [1,1,1,2,3],
        [1,1,1,2,3],
        [2,2,2,2,3],
        [3,3,3,3,3],
        [4,4,4,4,4]
    ]).astype(np.float32)


In [ ]:
conv = torch.nn.Conv2d(
    in_channels=1,
    out_channels=1,
    kernel_size=3,
    bias=False,
    stride = 1,
    padding_mode='zeros',
    padding=0
)

X_tensor = torch.from_numpy(X.reshape(1,1,6,5))
X_tensor.requires_grad = True
conv.weight = torch.nn.Parameter(torch.from_numpy(W.reshape(1,1,3,3)))
Z_tensor = conv(X_tensor)

print(Z_tensor)


tensor([[[[ 0., -4., -8.],
          [ 0., -3., -7.],
          [ 0., -1., -4.],
          [ 0.,  0., -1.]]]], grad_fn=<ConvolutionBackward0>)


In [ ]:
Z = conv2d(X,W,stride=1,padding=0)
print(Z)

[[ 0. -4. -8.]
 [ 0. -3. -7.]
 [ 0. -1. -4.]
 [ 0.  0. -1.]]


In [ ]:
loss = Z_tensor.sum()
loss.backward()

print(conv.weight.grad)
print(X_tensor.grad)


tensor([[[[15., 18., 25.],
          [21., 23., 28.],
          [30., 31., 34.]]]])
tensor([[[[ 1.,  1.,  0., -1., -1.],
          [ 3.,  3.,  0., -3., -3.],
          [ 4.,  4.,  0., -4., -4.],
          [ 4.,  4.,  0., -4., -4.],
          [ 3.,  3.,  0., -3., -3.],
          [ 1.,  1.,  0., -1., -1.]]]])


In [ ]:
L = Z.sum()
Z_grad = np.ones_like(Z)

W_grad = conv2d(X,Z_grad,stride=1,padding=0)
print(W_grad)

Z_grad_p = np.pad(Z_grad, ((2, 2), (2, 2)), mode='constant')
Wf = np.flip(W, axis=(0, 1))

X_grad = conv2d(Z_grad_p,Wf,stride=1,padding=0)
print(X_grad)

[[15. 18. 25.]
 [21. 23. 28.]
 [30. 31. 34.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
[[ 1.  1.  0. -1. -1.]
 [ 3.  3.  0. -3. -3.]
 [ 4.  4.  0. -4. -4.]
 [ 4.  4.  0. -4. -4.]
 [ 3.  3.  0. -3. -3.]
 [ 1.  1.  0. -1. -1.]]
